## Audio Monitor

Monitors the audio by continuously recording and plotting the recorded audio.

You will need to start osp running in a terminal.

In [ ]:
# make Jupyter use the whole width of the browser
from IPython.display import Image, display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
# Pull in all the necessary python modules
import plotly.graph_objects as go
from IPython.display import Audio
from osp_control import OspControl
import os
import json
import numpy as np
import time
import ipywidgets as widgets
import socket
import threading
import sys

In [ ]:
DEST_IP = "192.168.86.38"
# MY_IP = "127.0.0.1"
MY_IP = "192.168.86.26"

In [ ]:
# Connect to OSP process
osp = OspControl(DEST_IP)  # connects to a local process

# set the number of bands and mute mic
# osp.send({"method": "set", "data": {"num_bands": 11}})
osp.send_chan({"alpha": 0})

In [ ]:
def plot_fft(fig, title, out1, update=True):
    ftrans = 10 * np.log10(np.abs(np.fft.fft(out1)/len(out1)))
    outlen = len(out1)
    values = np.arange(int(outlen/2))
    period = outlen/32000
    frequencies = values/period

    if update:
        with fig.batch_update():
            fig.data[0].y = ftrans
            fig.data[0].x = frequencies
            fig.update_layout()
            return
        
   
    fig.add_trace(go.Scatter(x=frequencies, line_color='green', opacity=1, y=ftrans))

    fig.update_layout(title=title,
                    xaxis_title='Frequency',
                    yaxis_title='dbFS',
                    template='plotly_white')

In [ ]:
def plot_mono(fig, title, out1, lab1, rate=48000, update=True):

    # Create x values.
    x = np.array(range(len(out1)))/rate
    if update:
        with fig.batch_update():
            fig.data[0].y = out1
            fig.data[0].x = x
            fig.update_layout()
            return
    fig.add_trace(go.Scatter(x=x,
                             y=out1,
                             name=lab1,
                             opacity=1))
    fig.update_layout(title=title,
#                       yaxis_range=[-4,4],
                      xaxis_title='Time(sec)',
                      yaxis_title='Amplitude',
                      template='plotly_white')

In [ ]:
update = False
def monitor(afig, ffig, interval=2):
    global update
    PORT = 5001
    sample_size = 384 * 1000  # 384 bytes per ms. Save 1000ms
    
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server_address = (MY_IP, PORT)
    sock.bind(server_address)
    sock.listen(1)
    filename = f"tcp://{MY_IP}:{PORT}"
    osp.send_chan({'audio_rfile': filename, "audio_record": 1})
    data = b''
    done = False
    
    # Wait for a connection
    with out:
        print('waiting for a connection')
    connection, client_address = sock.accept()
    try:
        with out:
            print('connection from', client_address)

        # Receive the data in small chunks and retransmit it
        while True:
            while len(data) < sample_size:
                new_data = connection.recv(32768)
                if len(new_data) == 0:
                    done = True
                    break
                data += new_data
            if done:
                break
            data = np.frombuffer(data, dtype=np.float32)
            data = np.reshape(data, (2, -1), 'F')
            # plot
            plot_mono(afig, '', data[0], 'out', update=update)
            plot_fft(ffig, '',  data[0], update=update)
            if update == False:
                update = True
            data = b''
            new_data = connection.recv(128000) # discard
    finally:
        with out:
            print("Closing socket")
        # Clean up the connection
        connection.close()
        sock.close()

In [ ]:
start_b = widgets.ToggleButton(
    value=False,
    description='Start',
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Start',
    icon='play'
)
def start_cb(k):
    if start_b.value:
        start_b.description = 'Stop'
        start_b.button_style='danger'
        start_b.tooltip='Stop Analysis'
        start_b.icon='stop'
        threading.Thread(target=monitor, args=(audio_fig, fft_fig)).start()
        return
    osp.send_chan({"audio_record": 0})
    start_b.description = 'Start'
    start_b.button_style='success'
    start_b.tooltip='Start'
    start_b.icon='play'
    
start_b.observe(start_cb, names='value')  

In [ ]:
out = widgets.Output(layout={'border': '1px solid black'})
audio_fig = go.FigureWidget()
fft_fig = go.FigureWidget()
widgets.VBox([audio_fig, fft_fig, start_b, out])